In [1]:
params_appliance = {
    'kettle': {
        'windowlength': 599,
        'on_power_threshold': 2000,
        'max_on_power': 3998,
        'mean': 700,
        'std': 1000,
        's2s_length': 128,
        'houses': [1, 2],
        'channels': [10, 8],
        'train_build': [1],
        'test_build': 2,
    },
    'microwave': {
        'windowlength': 599,
        'on_power_threshold': 200,
        'max_on_power': 3969,
        'mean': 500,
        'std': 800,
        's2s_length': 128,
        'houses': [1, 2],
        'channels': [13, 15],
        'train_build': [1],
        'test_build': 2,
    },
    'fridge': {
        'windowlength': 599,
        'on_power_threshold': 50,
        'max_on_power': 3323,
        'mean': 200,
        'std': 400,
        's2s_length': 512,
        'houses': [1, 2],
        'channels': [12, 14],
        'train_build': [1],
        'test_build': 2,
    },
    
   # 'television': { 
   #     'windowlength': 599,  # You may keep or adjust this
   #     'on_power_threshold': 10,
   #     'max_on_power': ?,     # To be calculated from meter data
   #     'mean': ?,             # Requires usage data
   #     'std': ?,              # Requires usage data
   #     's2s_length': ?,       # Typical sequence length for s2s models
   #     'houses': [1],
   #     'channels': [7],
   #     'train_build': [1],
   #     'test_build': []
   # }

}


DATA_DIRECTORY = 'dataset/ukdale/'
SAVE_PATH = 'ukdale_seq2point/microwave/'
AGG_MEAN = 522
AGG_STD = 814
APPLIANCE_NAME = 'microwave'



mains_data = {
    "mean": 522,
    "std":  814        
    }

In [2]:
import numpy as np
import pandas as pd
import time
import os
import matplotlib.pyplot as plt
import pandas as pd


def load_dataframe(directory, building, channel, col_names=['time', 'data'], nrows=None):
    df = pd.read_table(directory + 'house_' + str(building) + '/' + 'channel_' +
                       str(channel) + '.dat',
                       sep="\s+",
                       nrows=nrows,
                       usecols=[0, 1],
                       names=col_names,
                       dtype={'time': str},
                       )
    return df



training_directory="refit/microwave/microwave_training_.csv"
validation_directory="refit/microwave/microwave_validation_H5.csv"
save_model_dir="saved_models/ukdale_models/"



class get_arguments:
    def __init__(self):
        self.data_dir = DATA_DIRECTORY
        self.appliance_name = APPLIANCE_NAME
        self.aggregate_mean = 522
        self.aggregate_std = 814
        self.save_path = SAVE_PATH
        self.batch_size = 1000
        self.crop = 10000
        self.prunning_algorithm = "threshold"
        self.network_type = "seq2point"
        self.epochs = 20
        self.input_window_length = 599
        self.validation_frequency = 1


args = get_arguments()
appliance_name = args.appliance_name

In [3]:


def main():

    start_time = time.time()
    sample_seconds = 8
    training_building_percent = 95
    validation_percent = 13
    nrows = None
    debug = False

    train = pd.DataFrame(columns=['aggregate', appliance_name])

    for h in params_appliance[appliance_name]['houses']:
        print('    ' + args.data_dir + 'house_' + str(h) + '/'
              + 'channel_' +
              str(params_appliance[appliance_name]['channels'][params_appliance[appliance_name]['houses'].index(h)]) +
              '.dat')

        mains_df = load_dataframe(args.data_dir, h, 1)
        app_df = load_dataframe(args.data_dir,
                                h,
                                params_appliance[appliance_name]['channels'][params_appliance[appliance_name]['houses'].index(h)],
                                col_names=['time', appliance_name]
                                )

        mains_df['time'] = pd.to_datetime(mains_df['time'], unit='s')
        mains_df.set_index('time', inplace=True)
        mains_df.columns = ['aggregate']
        #resample = mains_df.resample(str(sample_seconds) + 'S').mean()
        mains_df.reset_index(inplace=True)

        if debug:
            print("    mains_df:")
            print(mains_df.head())
            plt.plot(mains_df['time'], mains_df['aggregate'])
            plt.show()

        # Appliance
        app_df['time'] = pd.to_datetime(app_df['time'], unit='s')

        if debug:
            print("app_df:")
            print(app_df.head())
            plt.plot(app_df['time'], app_df[appliance_name])
            plt.show()

        # the timestamps of mains and appliance are not the same, we need to align them
        # 1. join the aggragte and appliance dataframes;
        # 2. interpolate the missing values;
        mains_df.set_index('time', inplace=True)
        app_df.set_index('time', inplace=True)

        df_align = mains_df.join(app_df, how='outer'). \
            resample(str(sample_seconds) + 'S').mean().fillna(method='backfill', limit=1)
        df_align = df_align.dropna()

        df_align.reset_index(inplace=True)

        del mains_df, app_df, df_align['time']

        if debug:
            # plot the dtaset
            print("df_align:")
            print(df_align.head())
            plt.plot(df_align['aggregate'].values)
            plt.plot(df_align[appliance_name].values)
            plt.show()

        # Normilization ----------------------------------------------------------------------------------------------
        mean = params_appliance[appliance_name]['mean']
        std = params_appliance[appliance_name]['std']

        df_align['aggregate'] = (df_align['aggregate'] - args.aggregate_mean) / args.aggregate_std
        df_align[appliance_name] = (df_align[appliance_name] - mean) / std

        if h == params_appliance[appliance_name]['test_build']:
            # Test CSV
            df_align.to_csv(args.save_path + appliance_name + '_test_.csv', mode='a', index=False, header=False)
            print("    Size of test set is {:.4f} M rows.".format(len(df_align) / 10 ** 6))
            continue

        
        train = pd.concat([train, df_align], ignore_index=True)
        del df_align

    # Crop dataset
    if training_building_percent is not 0:
        train.drop(train.index[-int((len(train)/100)*training_building_percent):], inplace=True)


    # Validation CSV
    val_len = int((len(train)/100)*validation_percent)
    val = train.tail(val_len)
    val.reset_index(drop=True, inplace=True)
    train.drop(train.index[-val_len:], inplace=True)
    # Validation CSV
    val.to_csv(args.save_path + appliance_name + '_validation_' + '.csv', mode='a', index=False, header=False)

    # Training CSV
    train.to_csv(args.save_path + appliance_name + '_training_.csv', mode='a', index=False, header=False)

    print("    Size of total training set is {:.4f} M rows.".format(len(train) / 10 ** 6))
    print("    Size of total validation set is {:.4f} M rows.".format(len(val) / 10 ** 6))
    del train, val


    print("\nPlease find files in: " + args.save_path)
    print("Total elapsed time: {:.2f} min.".format((time.time() - start_time) / 60))


if __name__ == '__main__':
    main()

<>:86: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:86: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\HP\AppData\Local\Temp\ipykernel_6124\840787682.py:86: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if training_building_percent is not 0:


    dataset/ukdale/house_1/channel_13.dat


C:\Users\HP\AppData\Local\Temp\ipykernel_6124\840787682.py:25: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  mains_df['time'] = pd.to_datetime(mains_df['time'], unit='s')
C:\Users\HP\AppData\Local\Temp\ipykernel_6124\840787682.py:38: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  app_df['time'] = pd.to_datetime(app_df['time'], unit='s')
C:\Users\HP\AppData\Local\Temp\ipykernel_6124\840787682.py:53: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 

    dataset/ukdale/house_2/channel_15.dat


C:\Users\HP\AppData\Local\Temp\ipykernel_6124\840787682.py:25: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  mains_df['time'] = pd.to_datetime(mains_df['time'], unit='s')
C:\Users\HP\AppData\Local\Temp\ipykernel_6124\840787682.py:38: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  app_df['time'] = pd.to_datetime(app_df['time'], unit='s')
C:\Users\HP\AppData\Local\Temp\ipykernel_6124\840787682.py:53: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 

    Size of test set is 1.2936 M rows.
    Size of total training set is 0.7261 M rows.
    Size of total validation set is 0.1085 M rows.

Please find files in: ukdale_seq2point/microwave/
Total elapsed time: 22.56 min.


In [7]:
nrows = 10**5
path = DATA_DIRECTORY 
save_path = SAVE_PATH
aggregate_std = AGG_MEAN 
aggregate_mean = AGG_STD

start_time = time.time()


def load(path, building, appliance, channel, nrows=None):
    # load csv
    file_name = path + 'house_' + str(building) + '/' + 'channel_' + str(channel) + '.dat'
    single_csv = pd.read_csv(file_name,
                             sep=' ',
                             #header=0,
                             names=['time', appliance],
                             dtype={'time': str, "appliance": int},
                             #parse_dates=['time'],
                             #date_parser=pd.to_datetime,
                             nrows=nrows,
                             usecols=[0, 1],
                             engine='python'
                             )
    return single_csv



print("Starting creating testset...")

for h in params_appliance[appliance_name]['houses']:

    print(path + 'house_' + str(h) + '/'
          + 'channel_' +
          str(params_appliance[appliance_name]['channels'][params_appliance[appliance_name]['houses'].index(h)]) +
          '.dat')

    agg_df = load(path,
                  h,
                  appliance_name,
                  1,
                  nrows=nrows,
                  )

    df = load(path,
              h,
              appliance_name,
              params_appliance[appliance_name]['channels'][params_appliance[appliance_name]['houses'].index(h)],
              nrows=nrows,
              )

    #for i in range(100):
    #    print(int(df['time'][i]) - int(agg_df['time'][i]))

    # Time conversion
    print(df.head())
    print(agg_df.head())
    df['time'] = pd.to_datetime(df['time'], unit='ms')
    agg_df['time'] = pd.to_datetime(agg_df['time'], unit='ms')
    print(agg_df.head())
    print(df.head())

    df['aggregate'] = agg_df[appliance_name]
    cols = df.columns.tolist()
    del cols[0]
    cols = cols[-1:] + cols[:-1]
    df = df[cols]

    print(df.head())


    # Re-sampling
    ind = pd.date_range(0,  periods=df.shape[0], freq='6S')
    df.set_index(ind, inplace=True, drop=True)
    resample = df.resample('8S')
    df = resample.mean()

    print(df.head())

    # Normalization
    df['aggregate'] = (df['aggregate'] - aggregate_mean) / aggregate_std
    df[appliance_name] = \
        (df[appliance_name] - params_appliance[appliance_name]['mean']) / params_appliance[appliance_name]['std']

    # Save
    df.to_csv(save_path + appliance_name + '_test_' + 'uk-dale_' + 'H' + str(h) + '.csv', index=False)

    print("Size of test set is {:.3f} M rows (House {:d})."
          .format(df.shape[0] / 10 ** 6, h))

    del df


print("\nNormalization parameters: ")
print("Mean and standard deviation values USED for AGGREGATE are:")
print("    Mean = {:d}, STD = {:d}".format(aggregate_mean, aggregate_std))

print('Mean and standard deviation values USED for ' + appliance_name + ' are:')
print("    Mean = {:d}, STD = {:d}"
      .format(params_appliance[appliance_name]['mean'], params_appliance[appliance_name]['std']))

print("\nPlease find files in: " + save_path)
tot = int(int(time.time() - start_time) / 60)
print("\nTotal elapsed time: " + str(tot) + ' min')





Starting creating testset...
dataset/ukdale/house_1/channel_13.dat
         time  microwave
0  1355523693          1
1  1355523699          1
2  1355523705          1
3  1355523711          1
4  1355523717          1
         time  microwave
0  1352500095        599
1  1352500101        582
2  1352500107        600
3  1352500113        586
4  1352500120        596


C:\Users\HP\AppData\Local\Temp\ipykernel_6448\500205128.py:57: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['time'] = pd.to_datetime(df['time'], unit='ms')
C:\Users\HP\AppData\Local\Temp\ipykernel_6448\500205128.py:58: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  agg_df['time'] = pd.to_datetime(agg_df['time'], unit='ms')


                     time  microwave
0 1970-01-16 15:41:40.095        599
1 1970-01-16 15:41:40.101        582
2 1970-01-16 15:41:40.107        600
3 1970-01-16 15:41:40.113        586
4 1970-01-16 15:41:40.120        596
                     time  microwave
0 1970-01-16 16:32:03.693          1
1 1970-01-16 16:32:03.699          1
2 1970-01-16 16:32:03.705          1
3 1970-01-16 16:32:03.711          1
4 1970-01-16 16:32:03.717          1
   aggregate  microwave
0        599          1
1        582          1
2        600          1
3        586          1
4        596          1
                     aggregate  microwave
1970-01-01 00:00:00      590.5        1.0
1970-01-01 00:00:08      600.0        1.0
1970-01-01 00:00:16      586.0        1.0
1970-01-01 00:00:24      588.5        1.0
1970-01-01 00:00:32      597.0        1.0


C:\Users\HP\AppData\Local\Temp\ipykernel_6448\500205128.py:72: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  ind = pd.date_range(0,  periods=df.shape[0], freq='6S')
C:\Users\HP\AppData\Local\Temp\ipykernel_6448\500205128.py:74: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  resample = df.resample('8S')


Size of test set is 0.075 M rows (House 1).
dataset/ukdale/house_2/channel_15.dat
         time  microwave
0  1369085319          0
1  1369085325          0
2  1369085331          0
3  1369085337          0
4  1369085344          0
         time  microwave
0  1361117854        340
1  1361117860        341
2  1361117866        347
3  1361117872        350
4  1361117878        342


C:\Users\HP\AppData\Local\Temp\ipykernel_6448\500205128.py:57: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['time'] = pd.to_datetime(df['time'], unit='ms')
C:\Users\HP\AppData\Local\Temp\ipykernel_6448\500205128.py:58: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  agg_df['time'] = pd.to_datetime(agg_df['time'], unit='ms')


                     time  microwave
0 1970-01-16 18:05:17.854        340
1 1970-01-16 18:05:17.860        341
2 1970-01-16 18:05:17.866        347
3 1970-01-16 18:05:17.872        350
4 1970-01-16 18:05:17.878        342
                     time  microwave
0 1970-01-16 20:18:05.319          0
1 1970-01-16 20:18:05.325          0
2 1970-01-16 20:18:05.331          0
3 1970-01-16 20:18:05.337          0
4 1970-01-16 20:18:05.344          0
   aggregate  microwave
0        340          0
1        341          0
2        347          0
3        350          0
4        342          0
                     aggregate  microwave
1970-01-01 00:00:00      340.5        0.0
1970-01-01 00:00:08      347.0        0.0
1970-01-01 00:00:16      350.0        0.0
1970-01-01 00:00:24      341.5        0.0
1970-01-01 00:00:32      343.0        0.0


C:\Users\HP\AppData\Local\Temp\ipykernel_6448\500205128.py:72: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  ind = pd.date_range(0,  periods=df.shape[0], freq='6S')
C:\Users\HP\AppData\Local\Temp\ipykernel_6448\500205128.py:74: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  resample = df.resample('8S')


Size of test set is 0.075 M rows (House 2).

Normalization parameters: 
Mean and standard deviation values USED for AGGREGATE are:
    Mean = 814, STD = 522
Mean and standard deviation values USED for microwave are:
    Mean = 500, STD = 800

Please find files in: ukdale_seq2point/microwave/

Total elapsed time: 0 min


Data Feeder

In [33]:
import numpy as np 
import pandas as pd 

# batch_size: the number of rows fed into the network at once.
# crop: the number of rows in the data set to be used in total.
# chunk_size: the number of lines to read from the file at once.

class TrainSlidingWindowGenerator():

    """Yields features and targets for training a ConvNet.

    Parameters:
    __file_name (string): The path where the training dataset is located.
    __batch_size (int): The size of each batch from the dataset to be processed.
    __chunk_size (int): The size of each chunk of data to be processed.
    __shuffle (bool): Whether the dataset should be shuffled before being returned.
    __offset (int):
    __crop (int): The number of rows of the dataset to return.
    __skip_rows (int): The number of rows of a dataset to skip before reading data.
    __ram_threshold (int): The maximum amount of RAM to utilise at a time.
    total_size (int): The number of rows read from the dataset.

    """

    def __init__(self, 
                file_name, 
                chunk_size, 
                shuffle, 
                offset, 
                batch_size=1000, 
                crop=100000, 
                skip_rows=0, 
                ram_threshold=5 * 10 ** 5):
        self.__file_name = file_name
        self.__batch_size = batch_size
        self.__chunk_size = 10 ** 8
        self.__shuffle = shuffle
        self.__offset = offset
        self.__crop = crop
        self.__skip_rows = skip_rows
        self.__ram_threshold = ram_threshold
        self.__total_size = 0
        self.__total_num_samples = crop

    @property
    def total_num_samples(self):
        return self.__total_num_samples
    
    @total_num_samples.setter
    def total_num_samples(self, value):
        self.__total_num_samples = value

    def check_if_chunking(self):

        """Count the number of rows in the dataset and determine whether this is larger than the chunking 
        threshold or not. """

        # Loads the file and counts the number of rows it contains.
        print("Importing training file...")
        chunks = pd.read_csv(self.__file_name, 
                            header=0, 
                            nrows=self.__crop, 
                            skiprows=self.__skip_rows, skip_blank_lines=True)
        print("Counting number of rows...")
        self.__total_size = len(chunks)
        del chunks
        print("Done.")

        print("The dataset contains ", self.__total_size, " rows")

        # Display a warning if there are too many rows to fit in the designated amount RAM.
        if (self.__total_size > self.__ram_threshold):
            print("There is too much data to load into memory, so it will be loaded in chunks. Please note that this may result in decreased training times.")
    

    def load_dataset(self):

        """Yields pairs of features and targets that will be used directly by a neural network for training.

        Yields:
        input_data (numpy.array): A 1D array of size batch_size containing features of a single input. 
        output_data (numpy.array): A 1D array of size batch_size containing the target values corresponding to 
        each feature set.

        """

        if self.__total_size == 0:
            self.check_if_chunking()

        # If the data can be loaded in one go, don't skip any rows.
        if (self.__total_size <= self.__ram_threshold):

            # Returns an array of the content from the CSV file.
            data_array = np.array(pd.read_csv(self.__file_name, nrows=self.__crop, skiprows=self.__skip_rows, header=0, skip_blank_lines=True))
            inputs = data_array[:, 0]
            outputs = data_array[:, 1]

            maximum_batch_size = inputs.size - 2 * self.__offset
            self.total_num_samples = maximum_batch_size
            if self.__batch_size < 0:
                self.__batch_size = maximum_batch_size

            indicies = np.arange(maximum_batch_size)
            if self.__shuffle:
                np.random.shuffle(indicies)

            while True:
                for start_index in range(0, maximum_batch_size, self.__batch_size):
                    splice = indicies[start_index : start_index + self.__batch_size]
                    input_data = np.array([inputs[index : index + 2 * self.__offset + 1] for index in splice])
                    output_data = outputs[splice + self.__offset].reshape(-1, 1)

                    yield input_data, output_data
                    
        # Skip rows where needed to allow data to be loaded properly when there is not enough memory.
        if (self.__total_size >= self.__ram_threshold):
            number_of_chunks = np.arange(self.__total_size / self.__chunk_size)
            if self.__shuffle:
                np.random.shuffle(number_of_chunks)

            # Yield the data in sections.
            for index in number_of_chunks:
                data_array = np.array(pd.read_csv(self.__file_name, skiprows=int(index) * self.__chunk_size, header=0, nrows=self.__crop, skip_blank_lines=True))                   
                inputs = data_array[:, 0]
                outputs = data_array[:, 1]

                maximum_batch_size = inputs.size - 2 * self.__offset
                self.__total_num_samples = maximum_batch_size
                if self.__batch_size < 0:
                    self.__batch_size = maximum_batch_size

                indicies = np.arange(maximum_batch_size)
                if self.__shuffle:
                    np.random.shuffle(indicies)

            while True:
                for start_index in range(0, maximum_batch_size, self.__batch_size):
                    splice = indicies[start_index : start_index + self.__batch_size]
                    input_data = np.array([inputs[index : index + 2 * self.__offset + 1] for index in splice])
                    output_data = outputs[splice + self.__offset].reshape(-1, 1)

                    yield input_data, output_data
                    
class TestSlidingWindowGenerator(object):

    """Yields features and targets for testing and validating a ConvNet.

    Parameters:
    __number_of_windows (int): The number of sliding windows to produce.
    __offset (int): The offset of the infered value from the sliding window.
    __inputs (numpy.ndarray): The available testing / validation features.
    __targets (numpy.ndarray): The target values corresponding to __inputs.
    __total_size (int): The total number of inputs.

    """

    def __init__(self, number_of_windows, inputs, targets, offset):
        self.__number_of_windows = number_of_windows
        self.__offset = offset
        self.__inputs = inputs
        self.__targets = targets
        self.total_size = len(inputs)

    def load_dataset(self):

        """Yields features and targets for testing and validating a ConvNet.

        Yields:
        input_data (numpy.array): An array of features to test / validate the network with.

        """

        self.__inputs = self.__inputs.flatten()
        max_number_of_windows = self.__inputs.size - 2 * self.__offset

        if self.__number_of_windows < 0:
            self.__number_of_windows = max_number_of_windows

        indicies = np.arange(max_number_of_windows, dtype=int)
        for start_index in range(0, max_number_of_windows, self.__number_of_windows):
            splice = indicies[start_index : start_index + self.__number_of_windows]
            input_data = np.array([self.__inputs[index : index + 2 * self.__offset + 1] for index in splice])
            target_data = self.__targets[splice + self.__offset].reshape(-1, 1)
            yield input_data, target_data

Model

In [5]:
import tensorflow as tf 
import os

def create_model(input_window_length):

    """Specifies the structure of a seq2point model using Keras' functional API.

    Returns:
    model (tensorflow.keras.Model): The uncompiled seq2point model.

    """

    input_layer = tf.keras.layers.Input(shape=(input_window_length,))
    reshape_layer = tf.keras.layers.Reshape((1, input_window_length, 1))(input_layer)
    conv_layer_1 = tf.keras.layers.Convolution2D(filters=30, kernel_size=(10, 1), strides=(1, 1), padding="same", activation="relu")(reshape_layer)
    conv_layer_2 = tf.keras.layers.Convolution2D(filters=30, kernel_size=(8, 1), strides=(1, 1), padding="same", activation="relu")(conv_layer_1)
    conv_layer_3 = tf.keras.layers.Convolution2D(filters=40, kernel_size=(6, 1), strides=(1, 1), padding="same", activation="relu")(conv_layer_2)
    conv_layer_4 = tf.keras.layers.Convolution2D(filters=50, kernel_size=(5, 1), strides=(1, 1), padding="same", activation="relu")(conv_layer_3)
    conv_layer_5 = tf.keras.layers.Convolution2D(filters=50, kernel_size=(5, 1), strides=(1, 1), padding="same", activation="relu")(conv_layer_4)
    flatten_layer = tf.keras.layers.Flatten()(conv_layer_5)
    label_layer = tf.keras.layers.Dense(1024, activation="relu")(flatten_layer)
    output_layer = tf.keras.layers.Dense(1, activation="linear")(label_layer)

    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    return model

def save_model(model, network_type, algorithm, appliance, save_model_dir):

    """ Saves a model to a specified location. Models are named using a combination of their 
    target appliance, architecture, and pruning algorithm.

    Parameters:
    model (tensorflow.keras.Model): The Keras model to save.
    network_type (string): The architecture of the model ('', 'reduced', 'dropout', or 'reduced_dropout').
    algorithm (string): The pruning algorithm applied to the model.
    appliance (string): The appliance the model was trained with.

    """
    
    #model_path = "saved_models/" + appliance + "_" + algorithm + "_" + network_type + "_model.h5"
    model_path = save_model_dir

    if not os.path.exists (model_path):
        open((model_path), 'a').close()

    model.save(model_path)

def load_model(model, network_type, algorithm, appliance, saved_model_dir):

    """ Loads a model from a specified location.

    Parameters:
    model (tensorflow.keras.Model): The Keas model to which the loaded weights will be applied to.
    network_type (string): The architecture of the model ('', 'reduced', 'dropout', or 'reduced_dropout').
    algorithm (string): The pruning algorithm applied to the model.
    appliance (string): The appliance the model was trained with.

    """

    #model_name = "saved_models/" + appliance + "_" + algorithm + "_" + network_type + "_model.h5"
    model_name = saved_model_dir
    print("PATH NAME: ", model_name)

    model = tf.keras.models.load_model(model_name)
    num_of_weights = model.count_params()
    print("Loaded model with ", str(num_of_weights), " weights")
    return model

In [6]:
def remove_space(string):
    return string.replace(" ","")

In [7]:
import tensorflow as tf 
import numpy as np
import matplotlib.pyplot as plt


#tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
class Trainer():

    """ Used to train a seq2point model with or without pruning applied Supports 
    various alternative architectures. 
    
    Parameters:
    __appliance (string): The target appliance.
    __network_type (string): The architecture of the model.
    __batch_size (int): The number of rows per testing batch.
    __window_size (int): The size of eaech sliding window
    __window_offset (int): The offset of the inferred value from the sliding window.
    __max_chunk_size (int): The largest possible number of row per chunk.
    __validation_frequency (int): The number of epochs between model validation.
    __training_directory (string): The directory of the model's training file.
    __validation_directory (string): The directory of the model's validation file.
    __training_chunker (TrainSlidingWindowGenerator): A sliding window provider 
    that returns feature / target pairs. For training use only.
    __validation_chunker (TrainSlidingWindowGenerator): A sliding window provider 
    that returns feature / target pairs. For validation use only.
    
    """

    def __init__(self, appliance, batch_size, crop, network_type, 
                 training_directory, validation_directory, save_model_dir,
                 epochs=10, input_window_length=599, validation_frequency = 1,
                 patience=3, min_delta=1e-6, verbose=1):
        self.__appliance = appliance
        self.__algorithm = network_type
        self.__network_type = network_type
        self.__crop = crop
        self.__batch_size = batch_size
        self.__epochs = epochs
        self.__patience = patience
        self.__min_delta = min_delta
        self.__verbose = verbose
        self.__loss = "mse"
        self.__metrics = ["mse", "msle", "mae"]
        self.__learning_rate = 0.001
        self.__beta_1=0.9
        self.__beta_2=0.999
        self.__save_model_dir = save_model_dir

        self.__input_window_length = input_window_length
        self.__window_size = 2+self.__input_window_length
        self.__window_offset = int((0.5 * self.__window_size) - 1)
        self.__max_chunk_size = 5 * 10 ** 2
        self.__validation_frequency = validation_frequency
        self.__ram_threshold=5*10**5
        self.__skip_rows_train=10000000
        self.__validation_steps=100
        self.__skip_rows_val = 0

        # Directories of the training and validation files. Always has the structure 
        # ./dataset_management/refit/{appliance_name}/{appliance_name}_training_.csv for training or 
        # ./dataset_management/refit/{appliance_name}/{appliance_name}_validation_.csv
        self.__training_directory = training_directory
        self.__validation_directory = validation_directory

        self.__training_chunker = TrainSlidingWindowGenerator(file_name=self.__training_directory, 
                                        chunk_size=self.__max_chunk_size, 
                                        batch_size=self.__batch_size, 
                                        crop=self.__crop, shuffle=True,
                                        skip_rows=self.__skip_rows_train, 
                                        offset=self.__window_offset, 
                                        ram_threshold=self.__ram_threshold)
        self.__validation_chunker = TrainSlidingWindowGenerator(file_name=self.__validation_directory, 
                                            chunk_size=self.__max_chunk_size, 
                                            batch_size=self.__batch_size, 
                                            crop=self.__crop, 
                                            shuffle=True,
                                            skip_rows=self.__skip_rows_val, 
                                            offset=self.__window_offset, 
                                            ram_threshold=self.__ram_threshold)

    def train_model(self):

        """ Trains an energy disaggregation model using a user-selected pruning algorithm (default is no pruning). 
        Plots and saves the resulting model. """

        # Calculate the optimum steps per epoch.
        # self.__training_chunker.check_if_chunking()
        #steps_per_training_epoch = np.round(int(self.__training_chunker.total_size / self.__batch_size), decimals=0)
        steps_per_training_epoch = np.round(int(self.__training_chunker.total_num_samples / self.__batch_size), decimals=0)
        
        model = create_model(self.__input_window_length)

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.__learning_rate, beta_1=self.__beta_1, beta_2=self.__beta_2), loss=self.__loss, metrics=self.__metrics) 
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=self.__min_delta, patience=self.__patience, verbose=self.__verbose, mode="auto")

        ## can use checkpoint ###############################################
        # checkpoint_filepath = "checkpoint/housedata/refit/"+ self.__appliance + "/"
        # model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        #     filepath = checkpoint_filepath,
        #     monitor='val_loss',
        #     verbose=0,
        #     save_best_only=True,
        #     save_weights_only=False,
        #     mode='auto',
        #     save_freq='epoch')        
        #callbacks=[early_stopping, model_checkpoint_callback]
        ###################################################################

        callbacks=[early_stopping]
        
        training_history = self.default_train(model, callbacks, steps_per_training_epoch)

        training_history.history["val_loss"] = np.repeat(training_history.history["val_loss"], self.__validation_frequency)

        model.summary()
        save_model(model, self.__network_type, self.__algorithm, 
                   self.__appliance, self.__save_model_dir)

        self.plot_training_results(training_history)

    def default_train(self, model, callbacks, steps_per_training_epoch):

        """ The default training method the neural network will use. No pruning occurs.

        Parameters:
        model (tensorflow.keras.Model): The seq2point model being trained.
        early_stopping (tensorflow.keras.callbacks.EarlyStopping): An early stopping callback to 
        prevent overfitting.
        steps_per_training_epoch (int): The number of training steps to occur per epoch.

        Returns:
        training_history (numpy.ndarray): The error metrics and loss values that were calculated 
        at the end of each training epoch.

        """
        # ########### this is retired ##############################
        # training_history = model.fit_generator(self.__training_chunker.load_dataset(),
        #     steps_per_epoch=steps_per_training_epoch,
        #     epochs=1,
        #     verbose=1,
        #     validation_data = self.__validation_chunker.load_dataset(),
        #     validation_steps=100,
        #     validation_freq=self.__validation_frequency,
        #     callbacks=[early_stopping])
        ############################################################

        training_history = model.fit(self.__training_chunker.load_dataset(),                            
                      steps_per_epoch=int(steps_per_training_epoch),
                      epochs=self.__epochs,
                      verbose=self.__verbose,
                      callbacks=callbacks,
                      validation_data=self.__validation_chunker.load_dataset(),
                      validation_freq=self.__validation_frequency,
                      validation_steps=self.__validation_steps)

        return training_history

    def plot_training_results(self, training_history):

        """ Plots and saves a graph of training loss against epoch.

        Parameters:
        training_history (numpy.ndarray): A timeseries of loss against epoch count.

        """

        plt.plot(training_history.history["loss"], label="MSE (Training Loss)")
        plt.plot(training_history.history["val_loss"], label="MSE (Validation Loss)")
        plt.title('Training History')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend()

        #file_name = "./" + self.__appliance + "/saved_models/" + self.__appliance + "_" + self.__pruning_algorithm + "_" + self.__network_type + "_training_results.png"
        #plt.savefig(fname=file_name)

In [10]:
import os
import logging
import numpy as np 
import keras
import pandas as pd
import tensorflow as tf 
import time
import matplotlib.pyplot as plt

class Tester():

    """ Used to test and evaluate a pre-trained seq2point model with or without pruning applied. 
    
    Parameters:
    __appliance (string): The target appliance.
    __algorithm (string): The (pruning) algorithm the model was trained with.
    __network_type (string): The architecture of the model.
    __crop (int): The maximum number of rows of data to evaluate the model with.
    __batch_size (int): The number of rows per testing batch.
    __window_size (int): The size of eaech sliding window
    __window_offset (int): The offset of the inferred value from the sliding window.
    __test_directory (string): The directory of the test file for the model.
    
    """

    def __init__(self, appliance, algorithm, crop, batch_size, network_type,
                 test_directory, saved_model_dir, log_file_dir,
                 input_window_length):
        self.__appliance = appliance
        self.__algorithm = algorithm
        self.__network_type = network_type

        self.__crop = crop
        self.__batch_size = batch_size
        self._input_window_length = input_window_length
        self.__window_size = self._input_window_length + 2
        self.__window_offset = int(0.5 * self.__window_size - 1)
        self.__number_of_windows = 100

        self.__test_directory = test_directory
        self.__saved_model_dir = saved_model_dir

        self.__log_file = log_file_dir
        logging.basicConfig(filename=self.__log_file,level=logging.INFO)

    def test_model(self):

        """ Tests a fully-trained model using a sliding window generator as an input. Measures inference time, gathers, and 
        plots evaluationg metrics. """

        test_input, test_target = self.load_dataset(self.__test_directory)
        model = create_model(self._input_window_length)
        model = load_model(model, self.__network_type, self.__algorithm, 
                           self.__appliance, self.__saved_model_dir)

        test_generator = TestSlidingWindowGenerator(number_of_windows=self.__number_of_windows, inputs=test_input, targets=test_target, offset=self.__window_offset)

        # Calculate the optimum steps per epoch.
        steps_per_test_epoch = np.round(int(test_generator.total_size / self.__batch_size), decimals=0)

        # Test the model.
        start_time = time.time()
        testing_history = model.predict(x=test_generator.load_dataset(), steps=steps_per_test_epoch, verbose=2)

        end_time = time.time()
        test_time = end_time - start_time

        evaluation_metrics = model.evaluate(x=test_generator.load_dataset(), steps=steps_per_test_epoch)

        self.log_results(model, test_time, evaluation_metrics)
        self.plot_results(testing_history, test_input, test_target)


    def load_dataset(self, directory):
        """Loads the testing dataset from the location specified by file_name.

        Parameters:
        directory (string): The location at which the dataset is stored, concatenated with the file name.

        Returns:
        test_input (numpy.array): The first n (crop) features of the test dataset.
        test_target (numpy.array): The first n (crop) targets of the test dataset.

        """

        data_frame = pd.read_csv(directory, nrows=self.__crop, skiprows=0, header=0)
        test_input = np.round(np.array(data_frame.iloc[:, 0], float), 6)
        test_target = np.round(np.array(data_frame.iloc[self.__window_offset: -self.__window_offset, 1], float), 6)
        
        del data_frame
        return test_input, test_target

    def log_results(self, model, test_time, evaluation_metrics):

        """Logs the inference time, MAE and MSE of an evaluated model.

        Parameters:
        model (tf.keras.Model): The evaluated model.
        test_time (float): The time taken by the model to infer all required values.
        evaluation metrics (list): The MSE, MAE, and various compression ratios of the model.

        """

        inference_log = "Inference Time: " + str(test_time)
        logging.info(inference_log)

        metric_string = "MSE: ", str(evaluation_metrics[0]), " MAE: ", str(evaluation_metrics[3])
        logging.info(metric_string)

        self.count_pruned_weights(model)  

    def count_pruned_weights(self, model):

        """ Counts the total number of weights, pruned weights, and weights in convolutional 
        layers. Calculates the sparsity ratio of different layer types and logs these values.

        Parameters:
        model (tf.keras.Model): The evaluated model.

        """
        num_total_zeros = 0
        num_dense_zeros = 0
        num_dense_weights = 0
        num_conv_zeros = 0
        num_conv_weights = 0
        for layer in model.layers:
            if np.shape(layer.get_weights())[0] != 0:
                layer_weights = layer.get_weights()[0].flatten()

                if "conv" in layer.name:
                    num_conv_weights += np.size(layer_weights)
                    num_conv_zeros += np.count_nonzero(layer_weights==0)

                    num_total_zeros += np.size(layer_weights)
                else:
                    num_dense_weights += np.size(layer_weights)
                    num_dense_zeros += np.count_nonzero(layer_weights==0)

        conv_zeros_string = "CONV. ZEROS: " + str(num_conv_zeros)
        conv_weights_string = "CONV. WEIGHTS: " + str(num_conv_weights)
        conv_sparsity_ratio = "CONV. RATIO: " + str(num_conv_zeros / num_conv_weights)

        dense_weights_string = "DENSE WEIGHTS: " + str(num_dense_weights)
        dense_zeros_string = "DENSE ZEROS: " + str(num_dense_zeros)
        dense_sparsity_ratio = "DENSE RATIO: " + str(num_dense_zeros / num_dense_weights)

        total_zeros_string = "TOTAL ZEROS: " + str(num_total_zeros)
        total_weights_string = "TOTAL WEIGHTS: " + str(model.count_params())
        total_sparsity_ratio = "TOTAL RATIO: " + str(num_total_zeros / model.count_params())

        print("LOGGING PATH: ", self.__log_file)

        logging.info(conv_zeros_string)
        logging.info(conv_weights_string)
        logging.info(conv_sparsity_ratio)
        logging.info("")
        logging.info(dense_zeros_string)
        logging.info(dense_weights_string)
        logging.info(dense_sparsity_ratio)
        logging.info("")
        logging.info(total_zeros_string)
        logging.info(total_weights_string)
        logging.info(total_sparsity_ratio)

    def plot_results(self, testing_history, test_input, test_target):

        """ Generates and saves a plot of the testing history of the model against the (actual) 
        aggregate energy values and the true appliance values.

        Parameters:
        testing_history (numpy.ndarray): The series of values inferred by the model.
        test_input (numpy.ndarray): The aggregate energy data.
        test_target (numpy.ndarray): The true energy values of the appliance.

        """

        testing_history = ((testing_history * params_appliance[self.__appliance]["std"]) + params_appliance[self.__appliance]["mean"])
        test_target = ((test_target * params_appliance[self.__appliance]["std"]) + params_appliance[self.__appliance]["mean"])
        test_agg = (test_input.flatten() * mains_data["std"]) + mains_data["mean"]
        test_agg = test_agg[:testing_history.size]

        # Can't have negative energy readings - set any results below 0 to 0.
        test_target[test_target < 0] = 0
        testing_history[testing_history < 0] = 0
        test_input[test_input < 0] = 0

        # Plot testing outcomes against ground truth.
        plt.figure(1)
        plt.plot(test_agg[self.__window_offset: -self.__window_offset], label="Aggregate")
        plt.plot(test_target[:test_agg.size - (2 * self.__window_offset)], label="Ground Truth")
        plt.plot(testing_history[:test_agg.size - (2 * self.__window_offset)], label="Predicted")
        plt.title(self.__appliance + " " + self.__network_type + "(" + self.__algorithm + ")")
        plt.ylabel("Power Value (Watts)")
        plt.xlabel("Testing Window")
        plt.legend()

        #file_path = "./" + self.__appliance + "/saved_models/" + self.__appliance + "_" + self.__algorithm + "_" + self.__network_type + "_test_figure.png"
        #plt.savefig(fname=file_path)

        plt.show()

In [36]:
import argparse
# Allows a model to be trained from the terminal.


# Create Trainer instance
trainer = Trainer(
    appliance= args.appliance_name,
    batch_size=args.batch_size,
    crop=args.crop,
    network_type=args.network_type,
    training_directory = training_directory,
    validation_directory = validation_directory,
    save_model_dir=save_model_dir,
    epochs=args.epochs,
    input_window_length=args.input_window_length,
    validation_frequency=args.validation_frequency
    #patience=3,
    #min_delta= 1e-6,
    #verbose=1
)

# Train the model
trainer.train_model()


Importing training file...
Counting number of rows...
Done.
The dataset contains  10000  rows
Epoch 1/20


ResourceExhaustedError: Graph execution error:

Detected at node 'model_6/dense_12/MatMul' defined at (most recent call last):
    File "C:\Users\HP\anaconda3_1\envs\tensorflowgpu\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\HP\anaconda3_1\envs\tensorflowgpu\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\HP\anaconda3_1\envs\tensorflowgpu\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\HP\anaconda3_1\envs\tensorflowgpu\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\Users\HP\anaconda3_1\envs\tensorflowgpu\lib\site-packages\ipykernel\kernelapp.py", line 701, in start
      self.io_loop.start()
    File "C:\Users\HP\anaconda3_1\envs\tensorflowgpu\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
  

In [12]:
import os

file_path = "ukdale_seq2point/kettle/kettle_training_.csv"
if os.path.exists(file_path):
    print("File exists!")
else:
    print("File does not exist!")

File exists!


In [13]:
df = pd.read_csv("ukdale_seq2point/kettle/kettle_training_.csv")

In [24]:
df = pd.read_csv(training_directory)

In [25]:
df

,0.0945945945945946,-0.699
0,0.073710,-0.699
1,0.095823,-0.699
2,0.078624,-0.699
3,0.090909,-0.699
4,0.072482,-0.699
...,...,...
712133,-0.417690,-0.699
712134,-0.415848,-0.699
712135,-0.417690,-0.699
712136,-0.410319,-0.699
